<a target="_blank" href="https://colab.research.google.com/github/Takosaga/ai_group_project/blob/base_model_example/notebooks/base_model_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Example
This notebook is to show a baseline model to show how it works without training

In [13]:
pip install timm torch torchvision gradio


1120.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [1]:
import timm
import torch
from torchvision import transforms
from PIL import Image
import gradio as gr

In [3]:
# Step 1: Load a pre-trained model
model_name = "resnet50"  # You can choose other models from timm
model = timm.create_model(model_name, pretrained=True)
model.eval()  # Set to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [4]:
# Step 2: Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])

In [5]:

# Step 3: Load class labels
# Download ImageNet class labels
import urllib.request

url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
urllib.request.urlretrieve(url, "imagenet-simple-labels.json")


('imagenet-simple-labels.json', <http.client.HTTPMessage at 0x7392daf7f4a0>)

In [7]:

import json
with open("imagenet-simple-labels.json") as f:
    class_labels = json.load(f)


In [8]:

# Step 4: Define inference function
def predict(image):
    # Preprocess the image
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

    # Get top-5 predictions
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    result = {class_labels[catid]: float(prob) for catid, prob in zip(top5_catid, top5_prob)}
    return result

In [9]:

# Step 5: Set up Gradio Interface
gr_interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=5),
    title=f"Image Classification with {model_name.upper()}",
    description="Upload an image to see the top-5 predictions."
)


In [10]:

# Step 6: Launch Gradio App
gr_interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
